In [1]:
import os
import sys
import numpy as np
import pandas as pd
import collections
import itertools
from mirnylib import h5dict, genome
from hiclib import mapping, fragmentHiC
import cooler
from multiprocessing import Pool

hi
hello from new mapping


In [2]:
genome_db = genome.Genome('/home/zzeng/cloud_research/PengGroup/ZZeng/Data/Haihui/Tcf1/HiC-seq/Jun2016/exp/iterative_mapping/genome.fa')

In [3]:
import HTSeq
def iv_to_str(iv):
        return iv.chrom + ':' + str(iv.start) + '-' + str(iv.end)  
    
idx = pd.IndexSlice

In [4]:
peaks_dict = {}

Tcf1_peaks_bed_file = HTSeq.BED_Reader('processed/HiC_around_Tcf1_peaks/WT_CD8_Tcf1_peaks.bed')
for alt in Tcf1_peaks_bed_file:
    peaks_dict[iv_to_str(alt.iv)] = alt.iv

In [5]:
Tcf1_dependent_up_genes_dict = {}
Tcf1_dependent_up_genes_bed_file = HTSeq.BED_Reader('processed/HiC_around_Tcf1_peaks/diff_genes/Tcf1_dependent_up_gene_promoter_iv.bed')
for alt in Tcf1_dependent_up_genes_bed_file:
    Tcf1_dependent_up_genes_dict[alt.name] = alt.iv

In [6]:
Tcf1_independent_up_genes_dict = {}
Tcf1_independent_up_genes_bed_file = HTSeq.BED_Reader('processed/HiC_around_Tcf1_peaks/diff_genes/Tcf1_independent_up_gene_promoter_iv.bed')
for alt in Tcf1_independent_up_genes_bed_file:
    Tcf1_independent_up_genes_dict[alt.name] = alt.iv

In [7]:
os.chdir('/home/zzeng/cloud_research/PengGroup/ZZeng/Data/Haihui/Tcf1/HiC-seq/Combine_Jun_Jul_2016/iterative_mapping_3')

import numpy as np
import pandas as pd
import collections
import cooler

cutoff = 10e-5
sig_df = pd.read_csv('fit-hi-c/WT_CD8.spline_pass2.significances.txt.gz', header=0, sep='\t')
sig_df = sig_df[sig_df['q-value'] < cutoff]

mycooler = cooler.Cooler('WT_CD8-res-10K.cool')
bin_df = mycooler.bins()[:]
bin_df['mid'] = ((bin_df['start'] + bin_df['end']) / 2).astype(int)

bin_start_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_start_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'start']].values)

bin_end_dict = collections.defaultdict(lambda: {})
for chrom in mycooler.chromnames:
    bin_end_dict[chrom] = dict(bin_df.loc[bin_df.chrom == chrom, ['mid', 'end']].values)

sig_df['start1'] = sig_df.apply(lambda row: bin_start_dict[row['chr1']][row['fragmentMid1']], axis=1)
sig_df['start2'] = sig_df.apply(lambda row: bin_start_dict[row['chr2']][row['fragmentMid2']], axis=1)

sig_df['name'] = sig_df.index
sig_df['strand'] = '.'
sig_df['score'] = -np.log10(sig_df['p-value'])
sig_df.loc[sig_df.score == np.inf, 'score'] = 1000

chr_matrix_list = []
for chrom in mycooler.chromnames:
    chr_matrix = mycooler.matrix(balance=True, as_pixels=True, join=True).fetch(chrom).fillna(0)
    chr_matrix = chr_matrix[(abs(chr_matrix.start1 - chr_matrix.start2) >= 20000) & (abs(chr_matrix.start1 - chr_matrix.start2) <= 2000000)]
    chr_matrix_list.append(chr_matrix)

matrix_df = pd.concat(chr_matrix_list).set_index(['chrom1', 'start1', 'start2'])
sig_df['balanced_count'] = sig_df.apply(lambda row: matrix_df.loc[idx[row['chr1'], row['start1'], row['start2']], 'balanced'], axis=1)

WT_sig_df = sig_df

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: divide by zero encountered in log10


In [8]:
Tcf1_WT_interactions_df_list = []

score_cutoff = 0
cutoff_sig_df = WT_sig_df[WT_sig_df.score >= score_cutoff]
cutoff_sig_1_df = cutoff_sig_df.set_index(['chr1', 'start1', 'start2'])
cutoff_sig_2_df = cutoff_sig_df.set_index(['chr1', 'start2', 'start1'])
idx = pd.IndexSlice

for peak, peak_iv in peaks_dict.iteritems():
    peak_mid = peak_iv.start
    peak_bin_start = peak_mid / 10000 * 10000
    chrom = peak_iv.chrom
    
    try:
        peak_cutoff_sig_2_df = cutoff_sig_2_df.loc[idx[chrom, peak_bin_start],['score', 'balanced_count']]
        peak_cutoff_sig_2_df['interaction_site_bin_start'] = peak_cutoff_sig_2_df.index
        peak_cutoff_sig_2_df['Tcf1_binding_bin_start'] = peak_bin_start
        peak_cutoff_sig_2_df['chrom'] = chrom
        peak_cutoff_sig_2_df['Tcf1_binding_iv'] = peak
    except:
        peak_cutoff_sig_2_df = None
    
    try:
        peak_cutoff_sig_1_df = cutoff_sig_1_df.loc[idx[chrom, peak_bin_start],['score', 'balanced_count']]
        peak_cutoff_sig_1_df['interaction_site_bin_start'] = peak_cutoff_sig_1_df.index
        peak_cutoff_sig_1_df['Tcf1_binding_bin_start'] = peak_bin_start
        peak_cutoff_sig_1_df['chrom'] = chrom
        peak_cutoff_sig_1_df['Tcf1_binding_iv'] = peak
    except:
        peak_cutoff_sig_1_df = None
    
    if peak_cutoff_sig_1_df is not None or peak_cutoff_sig_2_df is not None:
        Tcf1_WT_interactions_df_list.append(pd.concat([peak_cutoff_sig_1_df, peak_cutoff_sig_2_df])) 
        

/usr/local/lib/python2.7/dist-packages/pandas/core/indexing.py:1310: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


In [9]:
Tcf1_WT_interactions_df = pd.concat(Tcf1_WT_interactions_df_list).loc[:,['Tcf1_binding_iv', 'chrom', 'Tcf1_binding_bin_start', 'interaction_site_bin_start', 'score', 'balanced_count']]
Tcf1_WT_interactions_df.rename(columns={'score': 'WT_CD8_score', 'balanced_count': 'WT_CD8_balanced_count'}, inplace=True)
#Tcf1_WT_interactions_df.set_index(['chrom', 'Tcf1_binding_bin_start', 'interaction_site_bin_start'], inplace=True)

In [10]:
all_genes_dict = {}

all_genes_bed_file = HTSeq.BED_Reader('/home/zzeng/cloud_research/PengGroup/ZZeng/Annotation/gene_iv/mm9/gene_promoter_1k_iv_unique.bed')
for alt in all_genes_bed_file:
    all_genes_dict[alt.name] = alt.iv

In [11]:
all_genes_tuples = []

for gene_id, gene_iv in all_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    all_genes_tuples.append((gene_chrom, gene_bin_start))
all_genes_tuples = set(all_genes_tuples)


Tcf1_dependent_up_genes_tuples = []

for gene_id, gene_iv in Tcf1_dependent_up_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    Tcf1_dependent_up_genes_tuples.append((gene_chrom, gene_bin_start))
Tcf1_dependent_up_genes_tuples = set(Tcf1_dependent_up_genes_tuples)


Tcf1_independent_up_genes_tuples = []

for gene_id, gene_iv in Tcf1_independent_up_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    Tcf1_independent_up_genes_tuples.append((gene_chrom, gene_bin_start))
Tcf1_independent_up_genes_tuples = set(Tcf1_independent_up_genes_tuples)

In [12]:
all_genes_tuples_dict = collections.defaultdict(lambda: [])

for gene_id, gene_iv in all_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    all_genes_tuples_dict[(gene_chrom, gene_bin_start)].append(gene_id)
    

Tcf1_dependent_up_genes_tuples_dict = collections.defaultdict(lambda: [])

for gene_id, gene_iv in Tcf1_dependent_up_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    Tcf1_dependent_up_genes_tuples_dict[(gene_chrom, gene_bin_start)].append(gene_id)

    
Tcf1_independent_up_genes_tuples_dict = collections.defaultdict(lambda: [])

for gene_id, gene_iv in Tcf1_independent_up_genes_dict.iteritems():
    gene_chrom = gene_iv.chrom

    gene_mid = (gene_iv.start + gene_iv.end) / 2
    gene_bin_start = gene_mid / 10000 * 10000
    
    Tcf1_independent_up_genes_tuples_dict[(gene_chrom, gene_bin_start)].append(gene_id)

In [13]:
Tcf1_WT_interactions_df.shape

(318063, 6)

In [51]:
top_Tcf1_WT_interactions_df = Tcf1_WT_interactions_df.sort_values(by='WT_CD8_score', ascending=False).iloc[:50000,:]

In [52]:
top_Tcf1_WT_interactions_df['all_gene_TSS_at_interaction_site'] = top_Tcf1_WT_interactions_df.apply(lambda row: True if (row['chrom'], row['interaction_site_bin_start']) in all_genes_tuples else False, axis=1)
top_Tcf1_WT_interactions_df['Tcf1_dependent_up_gene_TSS_at_interaction_site'] = top_Tcf1_WT_interactions_df.apply(lambda row: True if (row['chrom'], row['interaction_site_bin_start']) in Tcf1_dependent_up_genes_tuples else False, axis=1)
top_Tcf1_WT_interactions_df['Tcf1_independent_up_gene_TSS_at_interaction_site'] = top_Tcf1_WT_interactions_df.apply(lambda row: True if (row['chrom'], row['interaction_site_bin_start']) in Tcf1_independent_up_genes_tuples else False, axis=1)

In [53]:
print len(all_genes_tuples), top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.all_gene_TSS_at_interaction_site].shape[0]
print len(Tcf1_dependent_up_genes_tuples), top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.Tcf1_dependent_up_gene_TSS_at_interaction_site].shape[0]
print len(Tcf1_independent_up_genes_tuples), top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.Tcf1_independent_up_gene_TSS_at_interaction_site].shape[0]

20910 12773
92 75
44 46


In [54]:
import scipy

v1 = len(all_genes_tuples)
v2 = len(Tcf1_dependent_up_genes_tuples)
v3 = top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.all_gene_TSS_at_interaction_site].shape[0]
v4 = top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.Tcf1_dependent_up_gene_TSS_at_interaction_site].shape[0]

print '%.1e' % scipy.stats.chi2_contingency(np.array([[v1, v2], [v3, v4]]))[1]

v2 = len(Tcf1_independent_up_genes_tuples)
v4 = top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.Tcf1_independent_up_gene_TSS_at_interaction_site].shape[0]
print '%.1e' % scipy.stats.chi2_contingency(np.array([[v1, v2], [v3, v4]]))[1]

7.6e-02
1.4e-02


In [55]:
print len(set(reduce(lambda x, y: x + y, top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.all_gene_TSS_at_interaction_site].apply(lambda row: all_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), [])))
print len(set(reduce(lambda x, y: x + y, top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.Tcf1_dependent_up_gene_TSS_at_interaction_site].apply(lambda row: Tcf1_dependent_up_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), [])))
print len(set(reduce(lambda x, y: x + y, top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.Tcf1_independent_up_gene_TSS_at_interaction_site].apply(lambda row: Tcf1_independent_up_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), [])))

6776
32
20


In [56]:
v3 = len(set(reduce(lambda x, y: x + y, top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.all_gene_TSS_at_interaction_site].apply(lambda row: all_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), [])))

v2 = len(Tcf1_dependent_up_genes_tuples)
v4 = len(set(reduce(lambda x, y: x + y, top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.Tcf1_dependent_up_gene_TSS_at_interaction_site].apply(lambda row: Tcf1_dependent_up_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), [])))

print '%.1e' % scipy.stats.chi2_contingency(np.array([[v1, v2], [v3, v4]]))[1]

v2 = len(Tcf1_independent_up_genes_tuples)
v4 = len(set(reduce(lambda x, y: x + y, top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.Tcf1_independent_up_gene_TSS_at_interaction_site].apply(lambda row: Tcf1_independent_up_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), [])))

print '%.1e' % scipy.stats.chi2_contingency(np.array([[v1, v2], [v3, v4]]))[1]

8.1e-01
2.7e-01


In [43]:
print set(reduce(lambda x, y: x + y, top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.Tcf1_dependent_up_gene_TSS_at_interaction_site].apply(lambda row: Tcf1_dependent_up_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), []))

set(['Psrc1', 'Esco2', 'Kif4', 'Cep55', 'Cdca3', 'Ccnf', 'Iqgap3', 'Tpx2', 'Ppp1r1b', 'Stmn1', 'Bub1b', 'Aspm', 'Sgol1', 'Ska1', 'Rad51ap1', 'Prr11', '2810408I11Rik', 'Rad51', 'Tk1', 'Top2a', 'Tmem158', 'Ckap2l', 'Spc24', 'Neurl1b', 'Gpr15', 'Ccna2', 'Aurkb', 'Kif15', 'Kif11', 'Nusap1', 'S100a1', 'Sapcd2', '2810417H13Rik', 'Cdc25c', 'Birc5', 'Fam72a', 'Casc5', 'Troap', 'Kif2c'])


In [45]:
print set(reduce(lambda x, y: x + y, top_Tcf1_WT_interactions_df[top_Tcf1_WT_interactions_df.Tcf1_independent_up_gene_TSS_at_interaction_site].apply(lambda row: Tcf1_independent_up_genes_tuples_dict[(row['chrom'], row['interaction_site_bin_start'])], axis=1).tolist(), []))

set(['Trmt12', 'Pcgf2', 'D930048N14Rik', 'Rps27l', 'Espn', 'Myl10', 'Exoc8', 'Kcnj8', 'D630039A03Rik', 'Rsg1', 'Pcsk4', 'Nxnl1', 'Zfp759', 'Trim46', 'Gm4532', 'Dapk2', 'Lpar5'])


In [46]:
', '.join(['Trmt12', 'Pcgf2', 'D930048N14Rik', 'Rps27l', 'Espn', 'Myl10', 'Exoc8', 'Kcnj8', 'D630039A03Rik', 'Rsg1', 'Pcsk4', 'Nxnl1', 'Zfp759', 'Trim46', 'Gm4532', 'Dapk2', 'Lpar5'])

'Trmt12, Pcgf2, D930048N14Rik, Rps27l, Espn, Myl10, Exoc8, Kcnj8, D630039A03Rik, Rsg1, Pcsk4, Nxnl1, Zfp759, Trim46, Gm4532, Dapk2, Lpar5'

In [51]:
import scipy

In [69]:
scipy.stats.chi2_contingency(np.array([[20910, 44], [13509, 33]]))

(0.28184285078844556,
 0.5954958832956162,
 1,
 array([[ 20907.22767857,     46.77232143],
        [ 13511.77232143,     30.22767857]]))